In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier


In [2]:
orders = pd.read_csv('/content/orders.csv')
vendors = pd.read_csv('/content/vendors.csv')
customers = pd.read_csv('/content/train_customers.csv')
train_locations = pd.read_csv('/content/train_locations.csv')
test_customers = pd.read_csv('/content/test_customers.csv')
test_locations = pd.read_csv('/content/test_locations.csv')

/tmp/ipython-input-1805877687.py:1: DtypeWarning: Columns (15,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv('/content/orders.csv')


In [3]:
customers.isna().sum(), orders.isna().sum()

(customer_id        0
 gender         12154
 dob            31628
 status             0
 verified           0
 language       13575
 created_at         0
 updated_at         0
 dtype: int64,
 order_id                              0
 customer_id                           0
 item_count                         6526
 grand_total                           0
 payment_mode                          0
 promo_code                        61902
 vendor_discount_amount                0
 promo_code_discount_percentage    63800
 is_favorite                       34849
 is_rated                              0
 vendor_rating                     38139
 driver_rating                         0
 deliverydistance                      0
 preparationtime                   53377
 delivery_time                     61064
 order_accepted_time               46568
 driver_accepted_time              63800
 ready_for_pickup_time             47001
 picked_up_time                    46737
 delivered_time               

In [4]:
vendors.isna().sum(), train_locations.isna().sum()

(id                       0
 authentication_id        0
 latitude                 0
 longitude                0
 vendor_category_en       0
 vendor_category_id       0
 delivery_charge          0
 serving_distance         0
 is_open                  0
 OpeningTime              9
 OpeningTime2             9
 prepration_time          0
 commission              15
 is_haked_delivering      0
 discount_percentage      0
 status                   0
 verified                 0
 rank                     0
 language                15
 vendor_rating            0
 sunday_from_time1        1
 sunday_to_time1          1
 sunday_from_time2       58
 sunday_to_time2         58
 monday_from_time1        0
 monday_to_time1          0
 monday_from_time2       58
 monday_to_time2         58
 tuesday_from_time1       1
 tuesday_to_time1         1
 tuesday_from_time2      59
 tuesday_to_time2        59
 wednesday_from_time1     0
 wednesday_to_time1       0
 wednesday_from_time2    58
 wednesday_to_time2 

In [5]:
orders.head()

,order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163923,KL09J9N,6.0,10.1,1,NaN,0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,8/1/2024 5:30,84,8/2/2024 5:33,0,Work,KL09J9N X 0 X 84
1,163924,H5LGGFX,3.0,8.4,1,NaN,0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,8/1/2024 5:30,78,8/2/2024 5:34,0,Home,H5LGGFX X 0 X 78
2,163925,CYLZB6T,4.0,15.0,1,NaN,0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,8/1/2024 5:30,4,8/2/2024 5:35,0,Work,CYLZB6T X 0 X 4
3,163929,4YKUKYN,7.0,27.2,1,NaN,0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,8/1/2024 5:30,157,8/2/2024 5:39,0,Home,4YKUKYN X 0 X 157
4,163930,WDNU30K,1.0,6.5,1,NaN,0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,8/1/2024 5:30,160,8/2/2024 5:39,0,Home,WDNU30K X 0 X 160


In [6]:
vendors.head()

,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597,-0.588596,0.754434,Restaurants,2,0.0,6,1,11:00AM-11:30PM,...,1,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1,1,1/30/2023 14:42,4/7/2025 15:12,3,1
1,13,118608,-0.471654,0.744470,Restaurants,2,0.7,5,1,08:30AM-10:30PM,...,1,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1,1,5/3/2023 12:32,4/5/2025 20:46,3,1
2,20,118616,-0.407527,0.643681,Restaurants,2,0.0,8,1,08:00AM-10:45PM,...,1,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1,1,5/4/2023 22:28,4/7/2025 16:35,3,1
3,23,118619,-0.585385,0.753811,Restaurants,2,0.0,5,1,10:59AM-10:30PM,...,1,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1,1,5/6/2023 19:20,4/2/2025 0:56,3,1
4,28,118624,0.480602,0.552850,Restaurants,2,0.7,15,1,11:00AM-11:45PM,...,1,5,Burgers,Y,1,1,5/17/2023 22:12,4/5/2025 15:57,3,1


In [7]:
customers.head()

,customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2/7/2023 19:16,2/7/2023 19:16
1,ZGFSYCZ,Male,NaN,1,1,EN,2/9/2023 12:04,2/9/2023 12:04
2,S2ALZFL,Male,NaN,0,1,EN,3/14/2023 18:31,3/14/2023 18:31
3,952DBJQ,Male,NaN,1,1,EN,3/15/2023 19:47,3/15/2023 19:47
4,1IX6FXS,Male,NaN,1,1,EN,3/15/2023 19:57,3/15/2023 19:57


In [8]:
train_locations.head()

,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.682392,-78.789737
1,02SFNJH,1,NaN,1.679137,0.766823
2,02SFNJH,2,NaN,-0.498648,0.661241
3,RU43CXC,0,Home,0.100853,0.438165
4,BDFBPRD,0,NaN,2.523125,0.733464


In [9]:
test_locations.isna().sum(), test_customers.isna().sum()

(customer_id           0
 location_number       0
 location_type      7650
 latitude              3
 longitude             3
 dtype: int64,
 customer_id       0
 gender         3447
 dob            8920
 status            0
 verified          0
 language       3840
 created_at        0
 updated_at        0
 dtype: int64)

In [10]:
# 2. Clean Customers (train & test)
def clean_customers(df):
  df = df.copy()
  df['language'] = df['language'].fillna("Unknown")
  df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
  df['updated_at'] = pd.to_datetime(df['updated_at'], errors='coerce')
  ref_date = df['updated_at'].max()
  df['account_age_days'] = (ref_date - df['created_at']).dt.days
  return df[['customer_id','status','verified','language','account_age_days']]


cust_features = clean_customers(customers)
if test_customers is not None:
  test_cust_features = clean_customers(test_customers)

In [11]:
# 3. Clean Locations (train & test)
def clean_locations(df):
    df = df.copy()
    df['location_type'] = df['location_type'].fillna("NA")
    df = df.dropna(subset=['latitude','longitude'])
    return df[['customer_id','location_number','location_type','latitude','longitude']]


train_locations = clean_locations(train_locations)
if test_locations is not None:
  test_locations = clean_locations(test_locations)

In [12]:
# 4. Clean Vendors & Multi-hot Tags
vendors = vendors.copy()
for col in ['commission','vendor_rating']:
  vendors[col] = vendors[col].fillna(vendors[col].median())


vendors['vendor_tag_name'] = vendors['vendor_tag_name'].fillna("")
vendors['vendor_tag_list'] = vendors['vendor_tag_name'].apply(lambda x: x.split(",") if x != "" else [])


mlb = MultiLabelBinarizer()
vendor_tags_matrix = mlb.fit_transform(vendors['vendor_tag_list'])
tag_counts = vendor_tags_matrix.sum(axis=0)
top_tags_idx = np.argsort(tag_counts)[::-1][:50]
vendor_tags_df = pd.DataFrame(vendor_tags_matrix[:,top_tags_idx],
columns=[f"tag_{mlb.classes_[i]}" for i in top_tags_idx])


vendors = pd.concat([vendors.reset_index(drop=True), vendor_tags_df], axis=1)
vend_features = vendors[['id','latitude','longitude','vendor_category_id','vendor_category_en',
'delivery_charge','serving_distance','commission','vendor_rating'] + list(vendor_tags_df.columns)]

In [13]:
# 5. Orders → Aggregate per (customer,vendor)
orders = orders.copy()
for col in ['vendor_rating','driver_rating','item_count','grand_total','deliverydistance','preparationtime']:
    if col in orders.columns:
      orders[col] = orders[col].fillna(orders[col].median())
orders['created_at'] = pd.to_datetime(orders['created_at'], errors='coerce')


cv_agg = orders.groupby(['customer_id','vendor_id','LOCATION_NUMBER']).agg(
  cv_orders=('order_id','count'),
  cv_avg_items=('item_count','mean'),
  cv_avg_grand=('grand_total','mean'),
  cv_avg_distance=('deliverydistance','mean'),
  cv_avg_prep=('preparationtime','mean'),
  cv_avg_vend_rating=('vendor_rating','mean'),
  cv_avg_driver_rating=('driver_rating','mean'),
  last_order_date=('created_at','max')
).reset_index()


cutoff_date = orders['created_at'].max()
cv_agg['cv_last_order_days'] = (cutoff_date - cv_agg['last_order_date']).dt.days
cv_agg = cv_agg.drop(columns=['last_order_date'])

In [14]:
# 6. Candidate Generation Function (refined)
def build_candidates_refined(cust_features, locations, vendors, cv_agg, orders, top_k=50):
    # Customer-locations
    customer_locations = locations[['customer_id','location_number','location_type','latitude','longitude']].copy()
    customer_locations = customer_locations.rename(columns={'latitude':'cust_lat','longitude':'cust_long'})
    customer_locations['key'] = 1

    vendors_copy = vendors.copy()
    vendors_copy['key'] = 1

    # Cartesian join
    candidates = customer_locations.merge(vendors_copy, on='key').drop('key', axis=1)

    # Distance
    candidates['distance'] = np.sqrt((candidates['cust_lat'] - candidates['latitude'])**2 +
                                     (candidates['cust_long'] - candidates['longitude'])**2)

    # Top-K nearest vendors per customer-location
    candidates['rank_dist'] = candidates.groupby(['customer_id','location_number'])['distance'].rank(method='first')
    candidates = candidates[candidates['rank_dist'] <= top_k]

    # Historical interactions (ensure positives included)
    historical_pairs = orders[['customer_id','vendor_id']].drop_duplicates()
    candidates_hist = customer_locations.merge(
        historical_pairs, on='customer_id'
    ).merge(
        vendors_copy.drop(columns=['key']),
        left_on='vendor_id', right_on='id'
    )
    candidates_hist['distance'] = np.sqrt(
        (candidates_hist['cust_lat'] - candidates_hist['latitude'])**2 +
        (candidates_hist['cust_long'] - candidates_hist['longitude'])**2
    )

    # Combine top-K + historical
    candidates = pd.concat([candidates, candidates_hist], ignore_index=True)
    candidates = candidates.drop_duplicates(subset=['customer_id','location_number','id'])

    # Merge customer features
    candidates = candidates.merge(cust_features, on='customer_id', how='left')

    # Merge aggregated interactions
    candidates = candidates.merge(cv_agg, left_on=['customer_id','id'], right_on=['customer_id','vendor_id'], how='left')
    interaction_cols = [c for c in cv_agg.columns if c not in ['customer_id','vendor_id']]
    candidates[interaction_cols] = candidates[interaction_cols].fillna(0)

    # New binary feature: has_ordered_before
    candidates['has_ordered_before'] = (candidates['cv_orders'] > 0).astype(int)

    return candidates

# Build train & test candidates
train_candidates = build_candidates_refined(cust_features, train_locations, vend_features, cv_agg, orders, top_k=50)
if test_customers is not None and test_locations is not None:
    test_candidates = build_candidates_refined(test_cust_features, test_locations, vend_features, cv_agg, orders, top_k=50)

In [15]:
# 7. Target Creation
seen_df = orders[['customer_id','vendor_id']].drop_duplicates()
train_candidates = train_candidates.merge(
    seen_df.assign(target=1),
    left_on=['customer_id','id'],
    right_on=['customer_id','vendor_id'],
    how='left'
)
train_candidates['target'] = train_candidates['target'].fillna(0).astype(int)
train_candidates = train_candidates.drop(columns=['vendor_id'])

if test_customers is not None:
    test_candidates['target'] = 0

In [16]:
# 8. Feature Engineering (Label Encoding for categoricals)
categorical_cols = ['location_type','vendor_category_en','language']

for col in categorical_cols:
    for df in [train_candidates, test_candidates]:
        if col in df.columns:
            df[col] = df[col].astype('category').cat.codes

feature_cols = [c for c in train_candidates.columns if c not in ['customer_id','id','location_number','target']]
X_train = train_candidates[feature_cols]
y_train = train_candidates['target']
X_test = test_candidates[feature_cols]

# Fill numeric NaNs
numeric_cols = X_train.select_dtypes(include=np.number).columns
for col in numeric_cols:
    median_val = X_train[col].median()
    X_train[col] = X_train[col].fillna(median_val)
    X_test[col] = X_test[col].fillna(median_val)

/tmp/ipython-input-2229003145.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].fillna(median_val)
/tmp/ipython-input-2229003145.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = X_test[col].fillna(median_val)
/tmp/ipython-input-2229003145.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [17]:
import pandas as pd
import numpy as np
from xgboost import XGBRanker
import lightgbm as lgb
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import ndcg_score

TOP_N = 5
sample_frac = 0.4


# 1️ Sample for safe evaluation
groups = train_candidates[['customer_id','location_number']].drop_duplicates()
sample_groups = groups.sample(frac=sample_frac, random_state=42)
train_sample = train_candidates.merge(sample_groups, on=['customer_id','location_number'], how='inner')

X_sample = train_sample[feature_cols]
y_sample = train_sample["target"]

# Split sample into train/validation
train_sample['group'] = train_sample['customer_id'].astype(str) + "_" + train_sample['location_number'].astype(str)

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(X_sample, groups=train_sample['group']))

X_tr, X_val = X_sample.iloc[train_idx], X_sample.iloc[val_idx]
y_tr, y_val = y_sample.iloc[train_idx], y_sample.iloc[val_idx]

train_group_tr = train_sample.iloc[train_idx].groupby('group').size().to_list()
val_group = train_sample.iloc[val_idx].groupby('group').size().to_list()


In [18]:
# 9 Train XGBRanker on sample
xgb_ranker = XGBRanker(
    tree_method="hist",
    objective="rank:pairwise",
    eval_metric="ndcg",
    random_state=42,
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100
)
xgb_ranker.fit(X_tr, y_tr, group=train_group_tr, verbose=False)
pred_xgb_val = xgb_ranker.predict(X_val)

In [19]:
# 10 Train LightGBM Ranker on sample
lgb_train_tr = lgb.Dataset(X_tr, label=y_tr, group=train_group_tr)
lgb_params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_eval_at": [TOP_N],
    "learning_rate": 0.05,
    "num_leaves": 31,
    "min_data_in_leaf": 20,
    "max_depth": -1,
    "verbosity": -1,
    "random_state": 42
}
lgb_ranker = lgb.train(lgb_params, lgb_train_tr, num_boost_round=200)
pred_lgb_val = lgb_ranker.predict(X_val)

In [20]:
# 11 Evaluate NDCG@TOP_N
def compute_ndcg(y_true, y_score, groups, k=TOP_N):
    ndcgs = []
    start = 0
    for g in groups:
        y_true_g = y_true[start:start+g].to_numpy().reshape(1, -1)
        y_score_g = y_score[start:start+g].reshape(1, -1)
        ndcgs.append(ndcg_score(y_true_g, y_score_g, k=k))
        start += g
    return np.mean(ndcgs)

ndcg_xgb = compute_ndcg(y_val, pred_xgb_val, val_group, k=TOP_N)
ndcg_lgb = compute_ndcg(y_val, pred_lgb_val, val_group, k=TOP_N)

print(f"NDCG@{TOP_N} - XGBRanker (sample): {ndcg_xgb:.4f}")
print(f"NDCG@{TOP_N} - LightGBM (sample): {ndcg_lgb:.4f}")

NDCG@5 - XGBRanker (sample): 0.5130
NDCG@5 - LightGBM (sample): 0.5130


In [21]:
X_train_full = train_candidates[feature_cols].astype(np.float32)
y_train_full = train_candidates["target"].astype(np.float32)

# Group by customer_id + location_number
train_candidates['group'] = train_candidates['customer_id'].astype(str) + "_" + train_candidates['location_number'].astype(str)
full_train_group = train_candidates.groupby('group').size().to_list()

# 12 Train LightGBM LambdaMART on full dataset
# -----------------------------
lgb_train_full = lgb.Dataset(
    X_train_full,
    label=y_train_full,
    group=full_train_group
)

lgb_params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_eval_at": [TOP_N],
    "learning_rate": 0.05,
    "num_leaves": 40,
    "min_data_in_leaf": 20,
    "max_depth": -1,
    "verbosity": -1,
    "random_state": 42
}

lgb_ranker = lgb.train(
    lgb_params,
    lgb_train_full,
    num_boost_round=500
)


In [22]:
# 13 Predict on test
# -----------------------------
X_test_full = test_candidates[feature_cols].astype(np.float32)
test_candidates["score"] = lgb_ranker.predict(X_test_full)

test_candidates = (
    test_candidates.groupby(['customer_id', 'location_number'], group_keys=False)
    .apply(lambda x: x.sort_values("score", ascending=False)
           .assign(target=[1 if i < TOP_N else 0 for i in range(len(x))]))
)



/tmp/ipython-input-4201123725.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sort_values("score", ascending=False)


In [23]:
# 15 Prepare submission
test_candidates["CID X LOC_NUM X VENDOR"] = (
    test_candidates["customer_id"] + " X " +
    test_candidates["location_number"].astype(str) + " X " +
    test_candidates["id"].astype(str)
)
submission = test_candidates[["CID X LOC_NUM X VENDOR", "target"]]
submission.to_csv("submission_final.csv", index=False)

submission = submission[["CID X LOC_NUM X VENDOR", "target"]]

print(submission)

       CID X LOC_NUM X VENDOR  target
281415      000IPH5 X 0 X 161       1
281418      000IPH5 X 0 X 193       1
281437      000IPH5 X 0 X 459       1
281425      000IPH5 X 0 X 265       1
281420      000IPH5 X 0 X 196       1
...                       ...     ...
565244      ZZZW5ZR X 1 X 679       0
565225      ZZZW5ZR X 1 X 192       0
565214      ZZZW5ZR X 1 X 106       0
565216      ZZZW5ZR X 1 X 113       0
565209       ZZZW5ZR X 1 X 79       0

[836650 rows x 2 columns]


In [24]:
submission.to_csv("submission.csv", index=False)
from google.colab import files

files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>